In [43]:
from IPython.core.display import HTML
import jieba

class SearcherIIndex():
    """倒排索引文本搜索实现类
    
    用倒排索引
    利用Python的集合运算，来实现候选结果集之间交、并运算
    
    Attributes:
        index: 检索使用的倒排索引
        max_id: 当前索引的文档最大ID
        doc_list: 索引文档原文8
    """
    def __init__(self, docs_file): 
        """初始化，用文件中的文本行构建倒排索引
        
        Args:
            docs_file:包含带索引文档(文本)的文件名
            
        """
        self.index = dict()    
        self.max_id = 0
        self.doc_list = [] 
        
        with open(docs_file, 'r') as f:
            docs_data = f.read()
        
        for doc in docs_data.split():
            self.add_doc(doc)

    def add_doc(self, doc):
        """向索引中添加新文档
        
        Args:
            doc:待检索的文档(文本)
        
        Returns:
            新增文档ID
        """
        self.doc_list.append(doc)
        for term in list(jieba.cut_for_search(doc)):
            #构建和更新各Term对应的Posting(集合)
            if term in self.index: 
                self.index[term].add(self.max_id)
            else:
                self.index[term] = set([self.max_id])
        self.max_id += 1
        return self.max_id - 1
    
    def word_match(self, word):
        """从倒排索引中获取包含word的候选文档ID集合
        
        Args:
            word:待检索的词(短语)
            
        Returns：
            包含待检索词(短语)的文档ID集合
        """
        result = None
        for term in list(jieba.cut(word)):
            if result is None:
                result = self.index.get(term, set())
            else:
                result = result & self.index.get(term, set())
        if result is None:
            result = set()
        return result

    def conv_query(self, query):
        """将用户的查询转换成用eval可运行、返回结果ID集合的代码段
        
        Args:
            query:待转换的原始查询字符串
        
        Returns:
            转换完成可通过eval执行返回ID集合的代码段字符串
        """
        query_new_parts = []
        all_parts = list(jieba.cut(query))
        idx = 0
        cache = '' #缓存变量，用于回收分词过程被切开的短语片段
        count_parts = len(all_parts)
        while idx < count_parts:
            if all_parts[idx] == '(' or all_parts[idx] == ')':
                query_new_parts.append(all_parts[idx])
            elif all_parts[idx] == ' ':
                query_new_parts.append(' ')
            elif all_parts[idx] in ('and', 'AND', '+'):
                query_new_parts.append('&')
            elif all_parts[idx] in ('or', 'OR'):
                query_new_parts.append('|')
            elif all_parts[idx] in ('not', 'NOT', '-'):
                query_new_parts.append('-')
            elif (idx + 1 < count_parts #被分词切开的短语部分回收至缓存
                  and all_parts[idx+1] not in (' ', ')')): 
                cache += all_parts[idx]
            elif (idx + 2 < count_parts #处理词间空格的形式
                  and all_parts[idx+1] == " " 
                  and all_parts[idx+2] not in ('(', ')', 'and', 'AND', '+', 'or', 'OR', 'NOT', 'not', '+', '-', ' ')): 
                query_new_parts.append("self.word_match('{}') & ".format(all_parts[idx]))
                idx += 2
                continue
            else:
                query_new_parts.append("self.word_match('{}')".format(cache + all_parts[idx]))
                cache = '' #合并完成清空缓存
            idx += 1
        query_new = ''.join(query_new_parts)
        return query_new

    def highlighter(self, doc, word):
        """用word对doc进行HTML高亮
        
        Args:
            doc:需要高亮的文档
            word:要进行高亮的关键词(查询)
            
        Returns:
            返回对关键词(查询)进行高亮的文档
        """
        for part in list(jieba.cut(word)):
            #TODO(CHG):短语高亮需要先分词
            if part not in ('(', ')', 'and', 'AND', 'or', 'OR', 'NOT', 'not', ' '):
                doc = doc.replace(part, '<span style="color:red">{}</span>'.format(part))
        return doc

    def search(self, query):
        """用query进行查询返回结果文档列表
        
        Args:
            query:用户的(复合)布尔查询字符串
            
        Returns:
            复合查询要求的(高亮)文档结果列表
        """
        result = []
        query_new = self.conv_query(query)
        for did in eval(query_new):
            result.append(self.highlighter(self.doc_list[did], query))
        return result

In [44]:
import string

class SearcherIIndexVII(SearcherIIndex):
    """倒排索引文本搜索实现类(改进)
    
    自定义解析，保留英文片段，将中文片段多粒度分词处理
    
    Attributes:
        index: 检索使用的倒排索引
        max_id: 当前索引的文档最大ID
        doc_list: 索引文档原文
    """
    def parse_doc(self, doc):
        """对文档进行自定义解析，保留英文串，对中文串多粒度分词
        
        Args:
            doc:待解析的原始文档
        
        Returns:
            解析结果列表，元素是切分得到的term
        """
        result = []
        state_last = ''
        cache = ''
        for c in doc:
            state_c = c in string.ascii_letters \
                or c.isdigit() \
                or c in ('-', ':', '.')
            if c == ' ':
                if state_last:
                    result.append(cache)
                else:
                    result.extend(list(jieba.cut_for_search(cache)))
                result.append(' ')
                cache = ''
                state_last = '' 
            else:
                if state_c == state_last:
                    cache += c
                else:
                    if state_last != '':
                        if state_last:
                            result.append(cache)
                        else:
                            result.extend(list(jieba.cut_for_search(cache)))
                    cache = c
                state_last = state_c
        if cache:
            if state_last:
                result.append(cache)
            else:
                result.extend(list(jieba.cut_for_search(cache)))
        return result
    
    def add_doc(self, doc):
        """向索引中添加新文档(正常索引及ngram索引)
        
        Args:
            doc:待检索的文档(文本)
        
        Returns:
            新增文档ID
        """
        self.doc_list.append(doc)
        doc = doc.lower()
        for term in self.parse_doc(doc):
            #构建和更新各Term对应的Posting(集合)
            if term in self.index: 
                self.index[term].add(self.max_id)
            else:
                self.index[term] = set([self.max_id])
        
        #构建ngram索引(以二元为例)
        doclen = len(doc)
        for i in range(doclen-1):
            term = doc[i:i+2]
            if term in self.index_b: 
                self.index_b[term].add(self.max_id)
            else:
                self.index_b[term] = set([self.max_id])
                
        self.max_id += 1
        return self.max_id - 1
    
    def dumpIndex(self):
        """原样输出索引，用于检查索引构建结果
        
        Returns:
            对索引(字典结构)的Dump输出
        """
        print(self.index)
    
    def conv_query(self, query):
        """将用户的查询转换成用eval可运行、返回结果ID集合的代码段
        
        Args:
            query:待转换的原始查询字符串
        
        Returns:
            转换完成可通过eval执行返回ID集合的代码段字符串
        """
        query_new_parts = []
        all_parts = list(self.parse_query(query))
        idx = 0
        cache = '' #缓存变量，用于回收分词过程被切开的短语片段
        count_parts = len(all_parts)
        while idx < count_parts:
            if all_parts[idx][1:] == '(' or all_parts[idx][1:] == ')':
                query_new_parts.append(all_parts[idx][1:])
            elif all_parts[idx][1:] == ' ':
                query_new_parts.append(' ')
            elif all_parts[idx][1:] in ('and', 'AND', '+'):
                query_new_parts.append('&')
            elif all_parts[idx][1:] in ('or', 'OR'):
                query_new_parts.append('|')
            elif all_parts[idx][1:] in ('not', 'NOT', '-'):
                query_new_parts.append('-')
            elif (idx + 1 < count_parts #对连续的内容分段结果集合中间加”&“运算符
                  and all_parts[idx+1][1:] not in (' ', ')')): 
                query_new_parts.append("{} & ".format(self.conv_part(all_parts[idx])))
            elif (idx + 2 < count_parts #处理词间、词与符号间空格的情况
                  and all_parts[idx+1][1:] == " " 
                  and all_parts[idx+2][1:] not in (')', 'and', 'AND', '+', 'or', 'OR', 'NOT', 'not', '+', '-', ' ')): 
                query_new_parts.append("{} & ".format(self.conv_part(all_parts[idx])))
                idx += 2
                continue
            else:
                query_new_parts.append(self.conv_part(cache + all_parts[idx]))
                cache = '' #合并完成清空缓存
            idx += 1
        query_new = ''.join(query_new_parts)
        return query_new
    
    def term_match(self, term):
        """在索引里找到term对应的posting集合
        
        Args:
            term:要检索的词项
            
        Results:
            term对应的posting集合
        """
        return self.index.get(term, set()) 
    
    def conv_part(self, part):
        """将带有类别标记的解析结果段 转化为 eval能进行计算的代码段
        
        Args:
            part:带有类别标记的解析结果段
            
        Results:
            eval能进行计算的代码段字符串(调用 term_match() 进行计算)
        """
        flag = part[0]
        if flag == 'e':
            return "self.term_match('{}')".format(part[1:])
        elif flag == 'c':
            return "(self.term_match('{}'))".format(
                "') & self.term_match('".join(jieba.cut(part[1:])))
        elif flag == 'f':
            return "self.frag_match('{}')".format(part[1:])
    
    def highlighter(self, doc, query):
        """用query对doc进行HTML高亮
        
        Args:
            doc:需要高亮的文档
            word:要进行高亮的关键词(查询)
            
        Returns:
            返回对关键词(查询)进行高亮的文档
        """
        n = 0
        #生成要进行高亮的关键词串集合
        word_set = set()
        query = query.lower()
        query_parts = self.parse_query(query)
        for query_part in query_parts:
            if query_part[0] == 'e' or query_part[0] == 'f':
                word_set.add(query_part[1:])
                if len(query_part[1:]) > n:
                    n = len(query_part[1:])
            elif query_part[0] == 'c':
                if len(query_part[1:]) > 1:
                    for word in jieba.cut(query_part[1:]):
                        word_set.add(word)
                        if len(word) > n:
                            n = len(word)
        
        #遍历文档替换高亮关键词串
        doc_low = doc.lower()
        i = 0
        result = []
        while True:
            end_idx = i + n
            if end_idx > len(doc_low):
                end_idx = len(doc_low)
            for j in range(end_idx, i, -1):
                if doc_low[i:j] in word_set:
                    break
            if doc_low[i:j] in word_set:
                result.append(
                    '<span style="color:red">{}</span>'.format(doc[i:j]))
            else:
                result.append(doc_low[i:j])
            i = j
            if i == len(doc_low):
                break
                
        return ''.join(result)
    
    def __init__(self, docs_file): 
        """初始化，用文件中的文本行构建倒排索引
        
        Args:
            docs_file:包含带索引文档(文本)的文件名
            
        """
        self.index = dict() #标准倒排索引
        self.index_b = dict() #ngram索引
        self.max_id = 0
        self.doc_list = [] 
        
        with open(docs_file, 'r') as f:
            docs_data = f.read()
        
        for doc in docs_data.split():
            self.add_doc(doc)
            
    def frag_match(self, frag):
        """对片段frag用ngram索引实现原样搜索
        
        Args:
            frag:要原样搜索的字符串
            
        Results:
            片段原样搜索的结果(文档ID)集合
        """
        frag = frag.lower() #大小写归一化
        result = None
        doclen = len(frag)
        for i in range(doclen - 1):
            term = frag[i:i+2]
            if result is None:
                result = self.index_b.get(term, set())
            else:
                result = result & self.index_b.get(term, set())
        return result
    
    def get_char_type(self, c):
        """返回当前字符的类型(e,c,s,f,b)
        
        Args:
            c:要进行判断的单个字符
            
        Results:
            返回判断结果(前缀)：e为英文，c为中文，s为空格，f为引号，b为括号
        """
        result = 'c'
        if c in string.ascii_letters \
                or c.isdigit() \
                or c in ('-', ':', '.'):
            result = 'e'
        elif c == '"':
            result = 'f'
        elif c == ' ':
            result = 's'
        elif c in ('(', ')'):
            result = 'b'
        return result
    
    def parse_query(self, doc):
        """对查询进行自定义解析，保留英文串，对中文串原型插入
        
        Args:
            doc:待解析的原始文档
        
        Returns:
            解析结果列表，元素是带有串类型标记(首字符，e为英文，c为中文，s为空格，f为引号，b为括号)
            的切分term结果
        """
        doc = doc.lower() + ' ' #解决末位字符状态切换问题的小技巧
        result = []
        doclen = len(doc)
        i = 0
        while True:
            cur_char_type = self.get_char_type(doc[i])
            for j in range(i+1, doclen):
                if cur_char_type == 'f': #当前符号为引号，找下一个引号
                    if self.get_char_type(doc[j]) == 'f':
                        break
                elif self.get_char_type(doc[j]) != cur_char_type: #当前符号非引号，找下一个状态变化
                    break
            if cur_char_type == 's': #对多个空格连续出现的情况进行合并
                result.append('s ')
            elif cur_char_type == 'f': #对引号只提取引号内字符串
                result.append(cur_char_type + doc[i+1:j])
                j += 1
            else:
                result.append(cur_char_type + doc[i:j])
            i = j
            if i >= doclen - 1:
                break
        return result

In [30]:
query = '"七连胜"'
searcher = SearcherIIndexVII('titles.txt')
print(searcher.parse_query(query))
print(searcher.search(query))

['f七连胜']
['一边倒！中国女排3-0横扫美国取<span style="color:red">七连胜</span>', '一边倒！中国女排3-0横扫美国取<span style="color:red">七连胜</span>', '一边倒！中国女排3-0横扫美国取<span style="color:red">七连胜</span>', '一边倒！中国女排3-0横扫美国取<span style="color:red">七连胜</span>', '一边倒！中国女排3-0横扫美国取<span style="color:red">七连胜</span>']


In [31]:
searcher = SearcherIIndexVII('titles.txt')

query = '"mate"'
print(searcher.parse_query(query))
print(searcher.conv_query(query))
result = searcher.search(query)
if result:
    for doc in result:
        display(HTML(doc))
else:
    print('No result.')

['fmate']
self.frag_match('mate')


In [32]:
import nltk
print(nltk.corpus.words.words()[:10])

['A', 'a', 'aa', 'aal', 'aalii', 'aam', 'Aani', 'aardvark', 'aardwolf', 'Aaron']


In [42]:
from collections import Counter
import nltk

class Corrector():
    """用二元索引实现拼写校正
    
    Attributes:
        index_b: 检索使用的二元索引
        max_id: 当前索引的单词最大ID
        doc_list: 索引单词原文
    """
    def __init__(self): 
        """初始化，用NLTK的words词典构建倒排索引
        """
        self.index_b = dict() #ngram索引
        self.max_id = 0
        self.doc_list = [] 
        
        for doc in nltk.corpus.words.words():
            self.add_doc(doc)
            
    def add_doc(self, doc):
        """向索引中添加新词(单词的二元索引)
        
        Args:
            doc:待检索的单词
        
        Returns:
            新增单词ID
        """
        self.doc_list.append(doc)
        doc = doc.lower()
        
        #构建二元索引
        doclen = len(doc)
        for i in range(doclen-1):
            term = doc[i:i+2]
            if term in self.index_b: 
                self.index_b[term].append(self.max_id)
            else:
                self.index_b[term] = [self.max_id]
                
        self.max_id += 1
        return self.max_id - 1
    
    def correct(self, word, limit=5):
        """拼写校正函数
        
        Args:
            word:待校正的词
            limit:返回结果的最大条数，默认值为5
            
        Returns:
            最可能的校正单词列表
        """
        word = word.lower() #大小写归一化
        result = []
        docid_list = []
        doclen = len(word)
        for i in range(doclen - 1):
            term = word[i:i+2]
            docid_list += self.index_b.get(term, [])
        docid_counter = Counter(docid_list)
        count = 0
        for elem in docid_counter.most_common(300):
            cor_word = self.doc_list[elem[0]]
            if len(cor_word) >= doclen - 1 and len(cor_word) <= doclen + 1:
                result.append(cor_word)
                count += 1
                if count > limit:
                    break
        return result

In [43]:
cor = Corrector()

In [44]:
print(cor.correct('retrival'))

['retrieval', 'pretribal', 'rearrival', 'rerival', 'retreatal', 'retrial']


In [25]:
from collections import Counter

Counter([1,1,2,3,4])

Counter({1: 2, 2: 1, 3: 1, 4: 1})

In [23]:
%debug

> <ipython-input-13-c918f0d68a99>(168)conv_query()
    166                 cache = '' #合并完成清空缓存
    167             idx += 1
--> 168         query_new = ''.join(query_new_parts)
    169         return query_new
    170 



ipdb>  quit


In [63]:
doc = '华为Mate30采用安卓系统'

n = 6
word_set = set(
    ['华为' ,'安卓', '安卓系统'])

#正向最大分词
i = 0
result_f = []
while True:
    end_idx = i + n
    if end_idx > len(doc):
        end_idx = len(doc)
    for j in range(end_idx, i, -1):
        if doc[i:j] in word_set:
            break
    result_f.append(doc[i:j])
    i = j
    if i == len(doc):
        break
print('|'.join(result_f))

#逆向最大分词
i = len(doc)
result_b = []
while True:
    end_idx = i - n
    if end_idx < 0:
        end_idx = 0
    for j in range(end_idx, i):
#         print(j,i,doc[j:i])
        if doc[j:i] in word_set:
            break
    result_b.insert(0, doc[j:i])
    i = j
    if i == 0:
        break
print('|'.join(result_b))

华为|M|a|t|e|3|0|采|用|安卓系统
华为|M|a|t|e|3|0|采|用|安卓系统


In [64]:
#列表倒序的简单方法：切片法
[1, 2, 3, 4, 5][::-1]

[5, 4, 3, 2, 1]

In [2]:
import requests

In [3]:
url = 'http://news.163.com/rank/'
r = requests.get(url)

In [4]:
from lxml import etree

In [5]:
sel = etree.HTML(r.text)

In [6]:
nodes = sel.xpath('//td/a')

In [7]:
nodes[0].attrib['href']

'https://sports.163.com/19/1203/04/EVEPDFHV00058781.html'

In [8]:
news_list = []
for node in nodes:
    news_list.append([node.text, node.attrib['href']])

In [9]:
import csv

In [10]:
with open('163news.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerows(news_list)

In [22]:
!head 163news.csv

建筑女王扎哈最新遗作 198米世界最高中庭美哭了,https://home.163.com/19/1126/07/EUT172TP001081EI.html
河南小伙花50万回村建别墅 只因爸妈住不惯城里房,https://home.163.com/19/1126/07/EUT17B1R001081EI.html
南京一汤圆店将顾客吃剩汤圆回锅出售 监控全拍下,https://news.163.com/19/1126/07/EUT2JI150001899N.html
香港新民党主席谴责暴力 被上百暴徒围攻泼水指骂,https://news.163.com/19/1126/07/EUT44SOQ00018AOR.html
歌手张咪自曝已到癌症晚期 曾演唱《我爱你中国》,https://ent.163.com/19/1126/10/EUTD5EF000038FO9.html
赌王98岁生日因人太多累到睡着 原配子女无人探视,https://ent.163.com/19/1126/07/EUT3O7MM00038FO9.html
"公安局长落马牵出30余人 警方拒公开""买官者""被",https://news.163.com/19/1126/09/EUTAT0NI0001899N.html
西安维权奔驰女车主涉刑案?警方:案情复杂延长复核,https://news.163.com/19/1126/09/EUT99GPC0001899N.html
"""操场埋尸案""时任校长黄炳松等19名公职人员被处",https://news.163.com/19/1126/16/EUU00ALS0001899O.html
佛系大学生宿舍养锦鲤 迷信财运只爱喝茶弹曲,https://home.163.com/19/1126/07/EUT171MR001081EI.html


In [24]:
with open('163news.csv', 'r') as f:
    f_csv = csv.reader(f)
    for row in f_csv:
        print(row[0], row[1])

建筑女王扎哈最新遗作 198米世界最高中庭美哭了 https://home.163.com/19/1126/07/EUT172TP001081EI.html
河南小伙花50万回村建别墅 只因爸妈住不惯城里房 https://home.163.com/19/1126/07/EUT17B1R001081EI.html
南京一汤圆店将顾客吃剩汤圆回锅出售 监控全拍下 https://news.163.com/19/1126/07/EUT2JI150001899N.html
香港新民党主席谴责暴力 被上百暴徒围攻泼水指骂 https://news.163.com/19/1126/07/EUT44SOQ00018AOR.html
歌手张咪自曝已到癌症晚期 曾演唱《我爱你中国》 https://ent.163.com/19/1126/10/EUTD5EF000038FO9.html
赌王98岁生日因人太多累到睡着 原配子女无人探视 https://ent.163.com/19/1126/07/EUT3O7MM00038FO9.html
公安局长落马牵出30余人 警方拒公开"买官者"被 https://news.163.com/19/1126/09/EUTAT0NI0001899N.html
西安维权奔驰女车主涉刑案?警方:案情复杂延长复核 https://news.163.com/19/1126/09/EUT99GPC0001899N.html
"操场埋尸案"时任校长黄炳松等19名公职人员被处 https://news.163.com/19/1126/16/EUU00ALS0001899O.html
佛系大学生宿舍养锦鲤 迷信财运只爱喝茶弹曲 https://home.163.com/19/1126/07/EUT171MR001081EI.html
“中国第一富婆”家中被盗  熟人不偷金银却盗毛毯 https://home.163.com/19/1122/07/EUINKEIA001081EI.html
湖南八旬老人卖房隐居深山 18年如1日重修石殿 https://home.163.com/19/1121/07/EUG57F8U001081EI.html
贵阳荒岛现烂尾别墅群  屋内全是垃圾竟值上亿元 https://home.163.com/19/1120/07/EUDIQ

In [38]:
result_detail = []
for news_info in news_list[:5]:
    title = news_info[0]
    url = news_info[1]
    r = requests.get(url)
    sel = etree.HTML(r.text)
    nodes = sel.xpath("//div[@id='endText']/p/text()")
    content = ''.join(map(lambda x: x.strip(), nodes))
    result_detail.append([url, title, content])

In [39]:
print(result_detail)

[['https://sports.163.com/19/1203/04/EVEPDFHV00058781.html', '梅西获颁2019年金球奖 6夺荣耀超C罗成历史第', "网易体育12月3日报道：北京时间12月3日凌晨3点30分，2019年金球奖颁奖典礼在法国巴黎的沙特莱剧院进行。颁奖典礼上分量最重的金球奖由梅西获得。阿根廷人力压范戴克、C罗获得2019年金球奖，其个人职业生涯第六次获金球，成为史上第一人。范戴克获银球，C罗获铜球。：金球奖（Ballon d'Or）是由法国《队报》旗下杂志《法国足球》（France Football）自1956年起举办的授予每年度最优秀的欧洲足球运动员的奖项，也是世界足坛上最富盛名，影响力最大的足球奖项评选之一。奖项自成立起一年评选一次，分金球、银球和铜球。评审团由176名各国专业记者组成，每个国家有一名代表。每名评审员将在30人候选名单内按优先顺序为5名球员投票，获得投票的球员按选票顺序依次获得6分、4分、3分、2分和1分。最后按照总得分高低排出金、银、铜球的顺序。2010年到2015年间和FIFA合办，称为FIFA金球，2016年重新开始以《法国足球》的名义单独评选。梅西和C罗是金球奖有史以来获得次数最多的球员，两人都分别拿过5个金球。自2008年起，两人开始了长达10年的“二人转”垄断，但这一态势去年被打破。2018年金球奖属于皇马的克罗地亚中场莫德里奇，C罗排名第二获得银球，巴萨的法国前锋格里兹曼排在第三，梅西无缘三甲。2019年对于梅西而言是悲喜交加的一年，却也是他个人职业生涯重上高峰，值得铭记的一年。在2019年他和一起获得西甲冠军、西班牙超级杯冠军。这一年阿根廷人共打入54粒进球，贡献23次助攻，获得欧冠最佳射手、西甲射手王和助攻王、欧洲金靴、西甲最佳球员和欧足联年度最佳前锋，稍早还力压范戴克成为2019FIFA足球先生。德罗巴宣布了梅西的获奖，上届金球得主莫德里奇为梅西颁出金球奖杯。梅西在2009年、2010年、2011年、2012年、2015年曾经5次获得金球奖（2010年到2015年间为FIFA金球），这一次是他职业生涯第六次获得金球奖，他超越此前和他并列的C罗成为国际足球史上第一人。除了金球奖，颁奖现场还颁发出其他奖项。上届科帕奖得主姆巴佩颁出2019年科帕奖，效力于的德利赫特获得科帕奖（最优秀21岁以

In [73]:
class NewsIndex(SearcherIIndexVII):
    """新闻内容索引，实现对CSV文件的索引、对长文的高亮摘要、对新闻链接的支持
    """
    def __init__(self, docs_file): 
        """初始化，用CSV文件中的title+content构建倒排索引，将url保存在link_list里
        
        Args:
            docs_file:包含新闻URL、title和content的csv文件的文件名
            
        """
        self.index = dict() #标准倒排索引
        self.index_b = dict() #ngram索引
        self.max_id = 0
        self.doc_list = [] 
        self.link_list = []
        
        with open(docs_file, 'r') as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                self.add_doc(row[1] + row[2])
                self.link_list.append(row[0])

    def highlighter(self, doc, query):
        """用query对doc进行HTML高亮
        
        Args:
            doc:需要高亮的文档
            word:要进行高亮的关键词(查询)
            
        Returns:
            返回对关键词(查询)进行高亮的文档
        """
        n = 0
        #生成要进行高亮的关键词串集合
        word_set = set()
        query = query.lower()
        query_parts = self.parse_query(query)
        for query_part in query_parts:
            if query_part[0] == 'e' or query_part[0] == 'f':
                word_set.add(query_part[1:])
                if len(query_part[1:]) > n:
                    n = len(query_part[1:])
            elif query_part[0] == 'c':
                if len(query_part[1:]) > 1:
                    for word in jieba.cut(query_part[1:]):
                        word_set.add(word)
                        if len(word) > n:
                            n = len(word)
        
        #遍历文档替换高亮关键词串
        doc_low = doc.lower()
        i = 0
        result = []
        side_len = 10 #设置上下文保留的宽度
        last_end = 0 #上一个命中关键词的末位位置
        while True:
            end_idx = i + n
            if end_idx > len(doc_low):
                end_idx = len(doc_low)
            for j in range(end_idx, i, -1):
                if doc_low[i:j] in word_set:
                    break
            if doc_low[i:j] in word_set:
                #追加从上一个关键词末位到当前关键词开头的周围上下文
                if i - last_end > 2 * side_len:
                    result.append(doc[last_end:last_end+side_len])
                    result.append('...')
                    result.append(doc[i-side_len:i])
                else:
                    result.append(doc[last_end:i])
                #追加高亮处理的关键词串
                result.append(
                    '<span style="color:red">{}</span>'.format(doc[i:j]))
                last_end = j
#             else:
#                 result.append(doc_low[i:j])
            i = j
            if i == len(doc_low):
                right_idx = last_end + side_len
                if right_idx > len(doc):
                    result.append(doc[last_end:])
                else:
                    result.append(doc[last_end:right_idx])
                    result.append('...')
                break
                
        return ''.join(result)
    
    def search(self, query):
        """用query进行查询返回结果文档列表
        
        Args:
            query:用户的(复合)布尔查询字符串
            
        Returns:
            复合查询要求的(高亮)文档结果列表
        """
        result = []
        query_new = self.conv_query(query)
        for did in eval(query_new):
            result.append([self.highlighter(self.doc_list[did], query), self.link_list[did]])
        return result

In [74]:
news_searcher = NewsIndex('163news_d.csv')

In [75]:
query = '梅西 "C罗"'
result = news_searcher.search(query)
if result:
    for doc,url in result:
        display(HTML('<a href="{}" target="_blank">{}</a>'.format(url,doc)))
else:
    print('No result.')

In [40]:
with open('163news_d.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerows(result_detail)

In [41]:
!head 163news_d.csv

https://sports.163.com/19/1203/04/EVEPDFHV00058781.html,梅西获颁2019年金球奖 6夺荣耀超C罗成历史第,"网易体育12月3日报道：北京时间12月3日凌晨3点30分，2019年金球奖颁奖典礼在法国巴黎的沙特莱剧院进行。颁奖典礼上分量最重的金球奖由梅西获得。阿根廷人力压范戴克、C罗获得2019年金球奖，其个人职业生涯第六次获金球，成为史上第一人。范戴克获银球，C罗获铜球。：金球奖（Ballon d'Or）是由法国《队报》旗下杂志《法国足球》（France Football）自1956年起举办的授予每年度最优秀的欧洲足球运动员的奖项，也是世界足坛上最富盛名，影响力最大的足球奖项评选之一。奖项自成立起一年评选一次，分金球、银球和铜球。评审团由176名各国专业记者组成，每个国家有一名代表。每名评审员将在30人候选名单内按优先顺序为5名球员投票，获得投票的球员按选票顺序依次获得6分、4分、3分、2分和1分。最后按照总得分高低排出金、银、铜球的顺序。2010年到2015年间和FIFA合办，称为FIFA金球，2016年重新开始以《法国足球》的名义单独评选。梅西和C罗是金球奖有史以来获得次数最多的球员，两人都分别拿过5个金球。自2008年起，两人开始了长达10年的“二人转”垄断，但这一态势去年被打破。2018年金球奖属于皇马的克罗地亚中场莫德里奇，C罗排名第二获得银球，巴萨的法国前锋格里兹曼排在第三，梅西无缘三甲。2019年对于梅西而言是悲喜交加的一年，却也是他个人职业生涯重上高峰，值得铭记的一年。在2019年他和一起获得西甲冠军、西班牙超级杯冠军。这一年阿根廷人共打入54粒进球，贡献23次助攻，获得欧冠最佳射手、西甲射手王和助攻王、欧洲金靴、西甲最佳球员和欧足联年度最佳前锋，稍早还力压范戴克成为2019FIFA足球先生。德罗巴宣布了梅西的获奖，上届金球得主莫德里奇为梅西颁出金球奖杯。梅西在2009年、2010年、2011年、2012年、2015年曾经5次获得金球奖（2010年到2015年间为FIFA金球），这一次是他职业生涯第六次获得金球奖，他超越此前和他并列的C罗成为国际足球史上第一人。除了金球奖，颁奖现场还颁发出其他奖项。上届科帕奖得主姆巴佩颁出2019年科帕奖，效力于的德利赫特获得科帕奖（最优秀21岁以下球员）。波兰前

In [15]:
url = 'https://sports.163.com/19/1203/04/EVEPDFHV00058781.html'
r = requests.get(url)
sel = etree.HTML(r.text)

In [28]:
nodes = sel.xpath("//div[@id='endText']/p/text()")

In [31]:
''.join(nodes)

"网易体育12月3日报道：北京时间12月3日凌晨3点30分，2019年金球奖颁奖典礼在法国巴黎的沙特莱剧院进行。颁奖典礼上分量最重的金球奖由梅西获得。阿根廷人力压范戴克、C罗获得2019年金球奖，其个人职业生涯第六次获金球，成为史上第一人。范戴克获银球，C罗获铜球。：金球奖（Ballon d'Or）是由法国《队报》旗下杂志《法国足球》（France Football）自1956年起举办的授予每年度最优秀的欧洲足球运动员的奖项，也是世界足坛上最富盛名，影响力最大的足球奖项评选之一。奖项自成立起一年评选一次，分金球、银球和铜球。评审团由176名各国专业记者组成，每个国家有一名代表。每名评审员将在30人候选名单内按优先顺序为5名球员投票，获得投票的球员按选票顺序依次获得6分、4分、3分、2分和1分。最后按照总得分高低排出金、银、铜球的顺序。2010年到2015年间和FIFA合办，称为FIFA金球，2016年重新开始以《法国足球》的名义单独评选。梅西和C罗是金球奖有史以来获得次数最多的球员，两人都分别拿过5个金球。自2008年起，两人开始了长达10年的“二人转”垄断，但这一态势去年被打破。2018年金球奖属于皇马的克罗地亚中场莫德里奇，C罗排名第二获得银球，巴萨的法国前锋格里兹曼排在第三，梅西无缘三甲。2019年对于梅西而言是悲喜交加的一年，却也是他个人职业生涯重上高峰，值得铭记的一年。在2019年他和一起获得西甲冠军、西班牙超级杯冠军。这一年阿根廷人共打入54粒进球，贡献23次助攻，获得欧冠最佳射手、西甲射手王和助攻王、欧洲金靴、西甲最佳球员和欧足联年度最佳前锋，稍早还力压范戴克成为2019FIFA足球先生。德罗巴宣布了梅西的获奖，上届金球得主莫德里奇为梅西颁出金球奖杯。梅西在2009年、2010年、2011年、2012年、2015年曾经5次获得金球奖（2010年到2015年间为FIFA金球），这一次是他职业生涯第六次获得金球奖，他超越此前和他并列的C罗成为国际足球史上第一人。除了金球奖，颁奖现场还颁发出其他奖项。上届科帕奖得主姆巴佩颁出2019年科帕奖，效力于的德利赫特获得科帕奖（最优秀21岁以下球员）。波兰前锋莱万多夫斯基作为颁奖嘉宾为的巴西门将阿利森颁出了首届雅辛奖（最佳门将）。美国女足运动员拉皮诺埃获得女足金球奖，她率领的美国队在女足世界杯夺冠，她本人也同时获得赛

In [20]:
for node in nodes:
    for t in node.itertext():
        print(t)


                    
                    
网易体育12月3日报道：
北京时间12月3日凌晨3点30分，2019年金球奖颁奖典礼在法国巴黎的沙特莱剧院进行。颁奖典礼上分量最重的金球奖由梅西获得。阿根廷人力压范戴克、C罗获得2019年金球奖，其个人职业生涯第六次获金球，成为史上第一人。范戴克获银球，C罗获铜球。
相关阅读
：
超惊险!梅西金球奖拿到686分 7分优势力压范戴克
第6座金球到货!梅西现场奖杯合体金光闪耀全场
梅西:10年前想不到今日成就 巴萨第1个人只排第2
C罗2年5次缺席重大颁奖礼 梅西:他没时间 这没什么
金球奖（Ballon d'Or）是由法国《队报》旗下杂志《法国足球》（France Football）自1956年起举办的授予每年度最优秀的欧洲足球运动员的奖项，也是世界足坛上最富盛名，影响力最大的足球奖项评选之一。奖项自成立起一年评选一次，分金球、银球和铜球。评审团由176名各国专业记者组成，每个国家有一名代表。每名评审员将在30人候选名单内按优先顺序为5名球员投票，获得投票的球员按选票顺序依次获得6分、4分、3分、2分和1分。最后按照总得分高低排出金、银、铜球的顺序。2010年到2015年间和FIFA合办，称为FIFA金球，2016年重新开始以《法国足球》的名义单独评选。
梅西和C罗是金球奖有史以来获得次数最多的球员，两人都分别拿过5个金球。自2008年起，两人开始了长达10年的“二人转”垄断，但这一态势去年被打破。2018年金球奖属于皇马的克罗地亚中场莫德里奇，C罗排名第二获得银球，巴萨的法国前锋格里兹曼排在第三，梅西无缘三甲。
2019年对于梅西而言是悲喜交加的一年，却也是他个人职业生涯重上高峰，值得铭记的一年。在2019年他和
巴塞罗那
一起获得西甲冠军、西班牙超级杯冠军。这一年阿根廷人共打入54粒进球，贡献23次助攻，获得欧冠最佳射手、西甲射手王和助攻王、欧洲金靴、西甲最佳球员和欧足联年度最佳前锋，稍早还力压范戴克成为2019FIFA足球先生。
德罗巴宣布了梅西的获奖，上届金球得主莫德里奇为梅西颁出金球奖杯。梅西在2009年、2010年、2011年、2012年、2015年曾经5次获得金球奖（2010年到2015年间为FIFA金球），这一次是他职业生涯第六次获得金球奖，他超越此前和他并列的C罗成